<a href="https://colab.research.google.com/github/tariqjamil-bwp/Coursera-Capstone-Project/blob/master/DL0321EN_4_1_Comparing_Models_py_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
## get the data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2022-03-20 05:29:21--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  26.6MB/s    in 9.2s    

2022-03-20 05:29:31 (27.2 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [2]:
!unzip -q -u concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

In [3]:
# removing .ipynb_checkpoints & .DS_Store, which are automatically created inside image folders.
# This will prevent errorneous image count in data_generator (i.e. 30001 instead of 30000 !!)

!find '.' -name '*.ipynb_checkpoints' -exec rm -r {} +
!find '.' -name '.DS_Store' -exec rm -r {} +

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [4]:
# importing liberaries

from keras.preprocessing.image import ImageDataGenerator
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense

from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [5]:
# declaring essential variables/constants

batch_size_training = 100
batch_size_validation = 100

num_classes = 2
image_resize = 224

In [6]:
# Construct an ImageDataGenerator for the training set and another one for the validation set.

data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,)

In [7]:
# VGG16 was originally trained on 224 × 224 images, so make sure to address that  when
# defining the ImageDataGenerator instances.

train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size = batch_size_training,
    class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size = batch_size_validation,
    class_mode='categorical')

Found 30000 images belonging to 2 classes.
Found 9500 images belonging to 2 classes.


In [8]:
# defining model

model_vgg16 = Sequential()

model_vgg16.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

58900480/58889256 [==============================] - 0s 0us/step


In [9]:
# adding Dense layer

model_vgg16.add(Dense(num_classes, activation='softmax'))

In [10]:
model_vgg16.layers

In [11]:
#model_vgg16.layers[0].layers

In [12]:
# prohibting model from modifying weights of vgg16 trained model funtional layers (convolutional + maxpool).

model_vgg16.layers[0].trainable = False

In [13]:
model_vgg16.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [14]:
model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [16]:
# Model training

fit_history = model_vgg16.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
300/300 [==============================] - 385s 1s/step - loss: 0.1409 - accuracy: 0.9511 - val_loss: 0.0368 - val_accuracy: 0.9918
Epoch 2/2
300/300 [==============================] - 365s 1s/step - loss: 0.0266 - accuracy: 0.9943 - val_loss: 0.0205 - val_accuracy: 0.9953


In [17]:
# saving model (used during solving the assignment, and future use..)

model_vgg16.save('classifier_vgg16_model.h5')

In [18]:
print(fit_history)
fit_history.history

{'accuracy': [0.9510666728019714, 0.9943333268165588],
 'loss': [0.14090219140052795, 0.026630496606230736],
 'val_accuracy': [0.9917894601821899, 0.9952631592750549],
 'val_loss': [0.036823831498622894, 0.020528893917798996]}

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [19]:
# Loading the saved model that was built using the ResNet50 model.

model_resnet = keras.models.load_model('classifier_resnet_model.h5')
#model_vgg16 = keras.models.load_model('classifier_vgg16_model.h5')   # for experimentation to avoid training again !!

In [20]:
# Construction of an ImageDataGenerator for the test set.

test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle = False,
    seed = 42
)

Found 500 images belonging to 2 classes.


In [21]:
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size

In [22]:
# evaluateing the models (vgg16) on the test data

metrics_vgg16 = model_vgg16.evaluate(test_generator, steps=STEP_SIZE_TEST, verbose=1)

# Model.evaluate_generator is replaced by Model.evaluate .. due deprication warning

15/15 [==============================] - 8s 289ms/step - loss: 0.0197 - accuracy: 0.9979


In [23]:
# evaluateing the models (resnet) on the test data

metrics_resnet = model_resnet.evaluate(test_generator, steps=STEP_SIZE_TEST, verbose=1)

# Model.evaluate_generator is replaced by Model.evaluate .. due deprication warning

15/15 [==============================] - 6s 225ms/step - loss: 0.0032 - accuracy: 1.0000


In [24]:
# performance of the classifier which used VGG16 pre-trained model.

print('VGG16 Model Performance on Test Data --> Loss={0:.4f}, Accuracy={1:0.4f}'.format(metrics_vgg16[0], metrics_vgg16[1]))

VGG16 Model Performance on Test Data --> Loss=0.0197, Accuracy=0.9979


In [25]:
# performance of the classifier which used ResNet pre-trained model.

print('RESNET Model Performance on Test Data --> Loss={0:.4f}, Accuracy={1:.4f}'.format(metrics_resnet[0], metrics_resnet[1]))

RESNET Model Performance on Test Data --> Loss=0.0032, Accuracy=1.0000


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [26]:
test_generator.reset()

In [27]:
# The prediction is based on the vgg16 model.

prediction_vgg16 = model_vgg16.predict(test_generator, steps=STEP_SIZE_TEST, verbose=1)

# model.predict_generator is replaced by model.predict .. due deprication warning

15/15 [==============================] - 4s 281ms/step


In [28]:
test_generator.reset()

In [29]:
# The prediction is based on the resnet model.

prediction_resnet = model_resnet.predict(test_generator, steps=STEP_SIZE_TEST, verbose=1)

# model.predict_generator is replaced by model.predict .. due deprication warning

15/15 [==============================] - 4s 216ms/step


In [30]:
# This function converts all predicted data into its equivalent class labels

def classify(predicted_data):
    predicted_class_indices=np.argmax(predicted_data,axis=1)
    labels = (train_generator.class_indices)
    labels = dict((v,k) for k,v in labels.items())
    predictions = [labels[k] for k in predicted_class_indices]
    return predictions

In [31]:
# Printing prediction results of first 5 images of test set using vgg16 model 

print("Prediction results on 1st 5 images of test set using VGG16 Model")
print("----------------------------------------------------------------")
      
for i in classify(prediction_vgg16[:5]):
    print(i)

Prediction results on 1st 5 images of test set using VGG16 Model
----------------------------------------------------------------
negative
negative
negative
negative
negative


In [32]:
# Printing prediction results of first 5 images of test set using resnet model 
print("Prediction results on 1st 5 images of test set using RESNET Model")
print("-----------------------------------------------------------------")

for i in classify(prediction_resnet[:5]):
    print(i)

Prediction results on 1st 5 images of test set using RESNET Model
-----------------------------------------------------------------
negative
negative
negative
negative
negative


In [33]:
test_generator.reset()
test_generator[0][1][:5]   ## verification of negative images

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

#### The result is all from same class as the test_data was generated from directory with 'shuffle=False' parameter.
#### This would have resulted the loading of test data in sequential order from folder of the first class.

### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).